In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
#present files
os.listdir()

['.ipynb_checkpoints',
 'rejection_data_full_mega.csv',
 'Mega_data.csv',
 'ml-to-predict-rejection.ipynb',
 'code_to_clean_the_raw_data.ipynb']

In [11]:
#how does the raw data look like
raw_data = pd.read_excel('1-08-2013.xlsx', header=[0,1,2])
# raw_data2 = raw_data.dropna()
raw_data.head(10)

Production summary                                                                \
                                                                                                                                               ID #   
                                                                                                                                 Unnamed: 0_level_2   
0                                                NaN                                                                                                  
1                                            13188-3                                                                                                  
2                                                NaN                                                                                                  
3                                                NaN                                                                                                  
4                                                NaN                                                                                                  
5                                                NaN                                                                                                  
6                                                NaN                                                                                                  
7                                            13191-1                                                                                                  
8                                            13193-1                                                                                                  
9                                            13195-1                                                                                                  

                                                                          \
        Construction Order Quantity                              Shrin.%   
  Unnamed: 1_level_2           F.F.  F.A         G.F. Unnamed: 5_level_2   
0                NaN            Yds  NaN          Yds                NaN   
1       30x30/110x62           5250  8.0  5706.521739               13.0   
2                NaN            NaN  NaN          NaN                NaN   
3                NaN            NaN  NaN          NaN                NaN   
4                NaN            NaN  NaN          NaN                NaN   
5                NaN            NaN  NaN          NaN                NaN   
6                NaN            NaN  NaN          NaN                NaN   
7       50x50/140x80           6750  8.0  7336.956522               12.0   
8                NaN           3500  8.0  3804.347826               12.0   
9                NaN           1250  8.0  1358.695652               12.0   

                                                          ...         \
             B.L.(m)                L #  Rec.             ...  TOTAL   
  Unnamed: 6_level_2 Unnamed: 7_level_2   B/L      B/L.1  ... Prodn.   
0                NaN                NaN   mtr        Yds  ...    Yds   
1        5997.824058               89.0  1750     1913.8  ...   1586   
2                NaN               81.0   860    940.496  ...    763   
3                NaN               89.0  1300    1421.68  ...   1160   
4                NaN              101.0   850     929.56  ...    769   
5                NaN              115.0  1270   1388.872  ...   1139   
6                NaN                NaN  6030   6594.408  ...   5417   
7        7623.857528               89.0  3000     3280.8  ...   2747   
8        3953.111311              117.0  2828  3092.7008  ...   2596   
9        1411.825468              108.0  2760   3018.336  ...   2562   

                                                      Date:- 01. August-13  \
              Shrin.% Grey(Inspected)Production        Unnamed: 16_level_1   
  Unnamed: 13_level_2

In [16]:
#total number of files in the directory
len(os.listdir())

273

In [ ]:
#primary data transforming

In [2]:
def data_clean(data_path):
    data = pd.read_excel(data_path, header=2)
    data = data.drop(0, axis=0)
    data = data.rename(columns={'Unnamed: 0':'ID','Unnamed: 1':'Construction', ' F.F.':'Req_Finish_Fabrics',
                                'F.A':'Fabric_Allowance', 'Unnamed: 5':'Shrink_allow', 
                      'B/L.1': 'Rec_Beam_length(yds)', 'Prodn..1':'Previous_pdn', 
                      'Prodn..2': 'Total_Pdn(yds)', 'Rej.&':'Rejection'})   
    data['Construction']= data['Construction'].fillna(method='ffill')
    data['Req_Finish_Fabrics']= data['Req_Finish_Fabrics'].fillna(method='ffill')
    data['Fabric_Allowance'] = data['Fabric_Allowance'].fillna(method ='ffill')
    data['Shrink_allow'] = data['Shrink_allow'].fillna(method ='ffill')
    data['Req_grey_fabric'] = data['Req_Finish_Fabrics']*100/(100-data['Fabric_Allowance'])
    data['Req_beam_length(yds)']= data['Req_grey_fabric']*100/(100-data['Shrink_allow'])/1.0936                         
    data = data[['ID', 'Construction','Req_Finish_Fabrics', 'Fabric_Allowance','Rec_Beam_length(yds)', 
                 'Shrink_allow', 'Previous_pdn','Req_grey_fabric','Req_beam_length(yds)', 'Total_Pdn(yds)', 'Rejection']]
    return data

In [ ]:
#combining all files 

In [3]:
def month_data_clean(month):
    p = [f'{i}.{month}.13.xlsx' for i in range(1,31)]
    df2 = []
    for i in p:
        if i in os.listdir():
            d = data_clean(i)
            df2.append(d)
        else:
            continue
    df = pd.concat(df2)
    return df

In [ ]:
#retrieving all data according to month

In [11]:
jan = month_data_clean('01')
feb = month_data_clean('02')
march = month_data_clean('03')
april = month_data_clean('04')
may = month_data_clean('05')
june = month_data_clean('06')
july = month_data_clean('07')
sept = month_data_clean('09')

In [ ]:
#the file names in august is in different form hence we made another code block

In [39]:
august_list = [f'{i}-08-2013.xlsx' for i in range(1,31)]
august = []
for i in august_list:
    if i in os.listdir():
        d = data_clean(i)
        august.append(d)
    else:
        continue

In [30]:
august = pd.concat(august)

In [ ]:
#combining all month data

In [ ]:
data = pd.concat([jan, feb, march,april, may, june, july, august, sept])

In [42]:
data.head(2)

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection
1,12207-8,40+40/2/40/110x80,31300.0,6.0,5752.336,12.5,5047,33297.87234,34797.651103,5047,NaN
2,NaN,40+40/2/40/110x80,31300.0,6.0,5883.568,12.5,1952,33297.87234,34797.651103,2111,NaN


In [ ]:
#the construction holds seveal information such as warp, weft count, epi, ppi hence we will retrieve it. 

In [ ]:
def construction_warp_count(x):
    try: 
        if '+' in x.split('/')[0]:
            return 'double'
        elif 'x' in x.split('/')[0]:
            return int(x.split('/')[0].split('x')[0])
    except :
        return 'N/A'

In [ ]:
def construction_weft_count(x): 
    try:
        return int(x.split('x')[1].split('/')[0])
    except Exception:
        if 'z40' in x.split('/')[1].split('x')[0]:
            return int(x.split('/')[1].split('x')[0].strip('z'))
    except:
         return int(x.split('x')[1].split('/')[0])

In [ ]:
data['warp_count']= data['Construction'].apply(lambda x: construction_warp_count(x))
data['weft_count']= data['Construction'].apply(lambda x: construction_weft_count(x))

In [249]:
def construction_to_epi(x):
    return x.split('/')[-1].split('x')[0]
def construction_to_ppi(x):
    try:
        return x.split('/')[-1].split('x')[1]
    except:
        return x.split('/')[-1]

In [224]:
data['epi']= data['Construction'].apply(lambda x: construction_to_epi(x))

In [ ]:
data['epi'] = data['epi'].replace('2', '140')

In [244]:
'60x60/156/100'.split('/')[-1]

['60x60', '156', '100']

In [256]:
data.head()

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi
1,12207-8,40+40/2/40/110x80,31300.0,6.0,5752.336,12.5,5047,33297.87234,34797.651103,5047,NaN,double,80.0,110,80
2,NaN,40+40/2/40/110x80,31300.0,6.0,5883.568,12.5,1952,33297.87234,34797.651103,2111,NaN,double,80.0,110,80
3,NaN,40+40/2/40/110x80,31300.0,6.0,3094.888,12.5,2207,33297.87234,34797.651103,2348,NaN,double,80.0,110,80
4,NaN,40+40/2/40/110x80,31300.0,6.0,5894.504,12.5,5026,33297.87234,34797.651103,5026,NaN,double,80.0,110,80
5,NaN,40+40/2/40/110x80,31300.0,6.0,5850.76,12.5,4391,33297.87234,34797.651103,4595,NaN,double,80.0,110,80


In [252]:
data['ppi']= data['Construction'].apply(lambda x: construction_to_ppi(x))

In [ ]:
#due to typing mistake we found some unusaul values and replace it with the correct ones

In [264]:
data['ppi'] = data['ppi'].replace('40100', '70')
data['ppi'] = data['ppi'].replace('40120', '80')
data['ppi'] = data['ppi'].replace('40140', '88')

In [270]:
data.shape

(121148, 15)

In [269]:
data[data['Previous_pdn']=='TOTAL'].shape

(22022, 15)

In [274]:
data.head(10)

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi
1,12207-8,40+40/2/40/110x80,31300.0,6.0,5752.336,12.5,5047,33297.87234,34797.651103,5047,NaN,double,80.0,110,80
2,NaN,40+40/2/40/110x80,31300.0,6.0,5883.568,12.5,1952,33297.87234,34797.651103,2111,NaN,double,80.0,110,80
3,NaN,40+40/2/40/110x80,31300.0,6.0,3094.888,12.5,2207,33297.87234,34797.651103,2348,NaN,double,80.0,110,80
4,NaN,40+40/2/40/110x80,31300.0,6.0,5894.504,12.5,5026,33297.87234,34797.651103,5026,NaN,double,80.0,110,80
5,NaN,40+40/2/40/110x80,31300.0,6.0,5850.76,12.5,4391,33297.87234,34797.651103,4595,NaN,double,80.0,110,80
6,NaN,40+40/2/40/110x80,31300.0,6.0,5905.44,12.5,4340,33297.87234,34797.651103,4565,NaN,double,80.0,110,80
7,NaN,40+40/2/40/110x80,31300.0,6.0,5905.44,12.5,3754,33297.87234,34797.651103,3954,NaN,double,80.0,110,80
8,NaN,40+40/2/40/110x80,31300.0,6.0,38286.936,12.5,TOTAL,33297.87234,34797.651103,27646,285,double,80.0,110,80
9,12228-1,40x40/110x90,10450.0,7.0,1443.552,14.5,1302,11236.55914,12017.350432,1302,NaN,40,40.0,110,90
10,NaN,40x40/110x90,10450.0,7.0,1749.76,14.5,1464,11236.55914,12017.350432,1464,NaN,40,40.0,110,90


In [ ]:
#it shows the rejection data involve against the proudct id hence to get rejection information we have to filter it

In [275]:
rejection_data = data[data['Previous_pdn']=='TOTAL']

In [276]:
rejection_data.shape

(22022, 15)

In [283]:
rejection_data['warp_count'] = rejection_data['warp_count'].fillna('double_80')

C:\Users\Toufique\AppData\Local\Temp/ipykernel_6844/3245873287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejection_data['warp_count'] = rejection_data['warp_count'].fillna('double_80')


In [288]:
rejection_data['warp_count'] = rejection_data['warp_count'].replace('N/A', 40)

C:\Users\Toufique\AppData\Local\Temp/ipykernel_6844/775827918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejection_data['warp_count'] = rejection_data['warp_count'].replace('N/A', 40)


In [291]:
rejection_data['warp_count'] = rejection_data['warp_count'].replace('double', 'double_40')

C:\Users\Toufique\AppData\Local\Temp/ipykernel_6844/374247729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejection_data['warp_count'] = rejection_data['warp_count'].replace('double', 'double_40')


In [295]:
rejection_data['warp_count'] = rejection_data['warp_count'].replace(501, 50)

C:\Users\Toufique\AppData\Local\Temp/ipykernel_6844/1449277692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejection_data['warp_count'] = rejection_data['warp_count'].replace(501, 50)


In [303]:
rejection_data['weft_count'] = rejection_data['weft_count'].fillna('40')

C:\Users\Toufique\AppData\Local\Temp/ipykernel_6844/2579525492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejection_data['weft_count'] = rejection_data['weft_count'].fillna('40')


In [310]:
rejection_data[rejection_data['Req_grey_fabric'].isna()]

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi
3,NaN,80/2x40/140x80,NaN,7.0,1235.768,10.0,TOTAL,NaN,NaN,114,0,double_80,40.0,140,80
3,NaN,40x40/110x70,NaN,7.0,328.08,10.0,TOTAL,NaN,NaN,165,2,40,40.0,110,70
3,NaN,40x40/130x76,NaN,10.5,355.42,15.5,TOTAL,NaN,NaN,74,0,40,40.0,130,76


In [311]:
rejection_data.shape

(22022, 15)

In [321]:
rejection_data['ID'] = rejection_data['ID'].fillna(method = 'ffill')

C:\Users\Toufique\AppData\Local\Temp/ipykernel_6844/2325500230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rejection_data['ID'] = rejection_data['ID'].fillna(method = 'ffill')


In [324]:
rejection_data[rejection_data['ID'].isna()].shape

(6063, 15)

In [337]:
rejection_all= rejection_data.drop('ID', axis=1)

In [341]:
rejection_all = rejection_all.dropna()

In [342]:
rejection_all.isna().sum()

Construction            0
Req_Finish_Fabrics      0
Fabric_Allowance        0
Rec_Beam_length(yds)    0
Shrink_allow            0
Previous_pdn            0
Req_grey_fabric         0
Req_beam_length(yds)    0
Total_Pdn(yds)          0
Rejection               0
warp_count              0
weft_count              0
epi                     0
ppi                     0
dtype: int64

In [344]:
rejection_all.to_csv('rejection_data_full.csv', index= False)

In [312]:
rejection_all = rejection_data.dropna()

In [327]:
rejection_all[rejection_all['ID'].isna()]

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi


In [328]:
rejection_all.isna().sum()

ID                      0
Construction            0
Req_Finish_Fabrics      0
Fabric_Allowance        0
Rec_Beam_length(yds)    0
Shrink_allow            0
Previous_pdn            0
Req_grey_fabric         0
Req_beam_length(yds)    0
Total_Pdn(yds)          0
Rejection               0
warp_count              0
weft_count              0
epi                     0
ppi                     0
dtype: int64

In [329]:
rejection_all.shape

(9, 15)

In [325]:
rejection_data.isna().sum()

ID                      6063
Construction               0
Req_Finish_Fabrics         3
Fabric_Allowance           0
Rec_Beam_length(yds)       1
Shrink_allow               0
Previous_pdn               0
Req_grey_fabric            3
Req_beam_length(yds)       3
Total_Pdn(yds)             0
Rejection                  8
warp_count                 0
weft_count                 0
epi                        0
ppi                        0
dtype: int64

In [289]:
rejection_data[rejection_data['warp_count']=='double']

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi
8,NaN,40+40/2/40/110x80,31300.0,6.0,38286.936,12.5,TOTAL,33297.872340,34797.651103,27646,285,double,80.0,110,80
8,NaN,40+40/2/40/110x80,31300.0,6.0,38286.936,12.5,TOTAL,33297.872340,34797.651103,28644,285,double,80.0,110,80
8,NaN,40+40/2/40/110x80,31300.0,6.0,38286.936,12.5,TOTAL,33297.872340,34797.651103,29900,285,double,80.0,110,80
454,NaN,40+40/2x40/110x80,17250.0,6.0,5293.024,12.0,TOTAL,18351.063830,19068.655473,5,0,double,40.0,110,80
8,NaN,40+40/2/40/110x80,31300.0,6.0,38286.936,12.5,TOTAL,33297.872340,34797.651103,30255,285,double,80.0,110,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,NaN,40+40/2x40/100x70,1180.0,8.2,1470.892,12.2,TOTAL,1285.403050,1338.709387,933,13,double,40.0,100,70
344,NaN,40+4/2x40/100x70,2510.0,7.1,3040.208,10.0,TOTAL,2701.829925,2745.092584,719,53,double,40.0,100,70
347,NaN,40+40/2x40/100x70,1180.0,8.2,1470.892,12.2,TOTAL,1285.403050,1338.709387,1108,13,double,40.0,100,70
344,NaN,40+4/2x40/100x70,2510.0,7.1,3040.208,10.0,TOTAL,2701.829925,2745.092584,888,72,double,40.0,100,70


In [284]:
rejection_data['warp_count'].value_counts()

40           16733
50            2964
30             852
60             434
20             425
double_80      252
double         220
N/A             69
501             25
100             21
120             20
70               7
Name: warp_count, dtype: int64

In [285]:
rejection_data[rejection_data['warp_count']=='N/A']

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi
252,NaN,40xz40/120x80,20815.0,8.0,5522.68,12.5,TOTAL,22625.000000,23644.058940,109,0,N/A,NaN,120,80
260,NaN,40xz40/120x80,20815.0,8.0,8256.68,12.5,TOTAL,22625.000000,23644.058940,542,14,N/A,NaN,120,80
265,NaN,40xz40/120x80,20815.0,8.0,11072.7,12.5,TOTAL,22625.000000,23644.058940,1108,14,N/A,NaN,120,80
269,NaN,40xz40/120x80,20815.0,8.0,11072.7,12.5,TOTAL,22625.000000,23644.058940,1813,116,N/A,NaN,120,80
274,NaN,40xz40/120x80,20815.0,8.0,11072.7,12.5,TOTAL,22625.000000,23644.058940,2558,127,N/A,NaN,120,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,NaN,40xz40/120x80,700.0,14.1,27121.28,14.6,TOTAL,814.901048,872.546346,22500,284,N/A,NaN,120,80
54,NaN,40xz40/120x80,700.0,14.1,27121.28,14.6,TOTAL,814.901048,872.546346,22500,284,N/A,NaN,120,80
54,NaN,40xz40/120x80,700.0,14.1,27121.28,14.6,TOTAL,814.901048,872.546346,22500,284,N/A,NaN,120,80
54,NaN,40xz40/120x80,700.0,14.1,27121.28,14.6,TOTAL,814.901048,872.546346,22500,284,N/A,NaN,120,80


In [265]:
data['ppi'].value_counts()

90     58903
80     32787
70     17249
100     4431
64      1473
76      1245
96       724
62       701
72       676
88       630
84       572
94       490
106      392
50       252
98       194
60       102
110       96
44        48
42        48
86        48
71        42
48        30
45        15
Name: ppi, dtype: int64

In [267]:
data[data['ppi']=='42']

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi
501,SF-13150,40x20/2/100x42,1000.0,10.2,147.636,17.0,0,1113.585746,1226.837576,23,NaN,20,20.0,100,42
502,NaN,40x20/2/100x42,1000.0,10.2,NaN,17.0,NaN,1113.585746,1226.837576,NaN,NaN,20,20.0,100,42
503,NaN,40x20/2/100x42,1000.0,10.2,147.636,17.0,TOTAL,1113.585746,1226.837576,23,0,20,20.0,100,42
522,SF-13150,40x20/2/100x42,1500.0,10.2,147.636,17.0,23,1670.378619,1840.256365,116,NaN,20,20.0,100,42
523,NaN,40x20/2/100x42,1500.0,10.2,NaN,17.0,NaN,1670.378619,1840.256365,NaN,NaN,20,20.0,100,42
524,NaN,40x20/2/100x42,1500.0,10.2,147.636,17.0,TOTAL,1670.378619,1840.256365,116,0,20,20.0,100,42
550,SF-13150,40x20/2/100x42,5588.0,10.2,147.636,17.0,116,6222.717149,6855.568377,116,NaN,20,20.0,100,42
551,NaN,40x20/2/100x42,5588.0,10.2,NaN,17.0,NaN,6222.717149,6855.568377,NaN,NaN,20,20.0,100,42
552,NaN,40x20/2/100x42,5588.0,10.2,147.636,17.0,TOTAL,6222.717149,6855.568377,116,0,20,20.0,100,42
573,SF-13150,40x20/2/100x42,5588.0,10.2,147.636,17.0,116,6222.717149,6855.568377,116,NaN,20,20.0,100,42


In [235]:
data['epi'].value_counts()

110    52810
120    31607
130    18755
100     5824
140     5284
156     1978
82      1473
160     1011
144      819
92       657
133      528
40       138
150      114
30        78
170       72
Name: epi, dtype: int64

In [231]:
'80/2x40140x88'.split('/')[-1].split('x')

['2', '40140', '88']

In [237]:
data[data['epi']=='30']

,ID,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi
288,13096-2,40x40/30x90,3000.0,8.3,1749.76,9.1,0,3271.537623,3291.012518,32,NaN,40,40.0,30
289,NaN,40x40/30x90,3000.0,8.3,NaN,9.1,NaN,3271.537623,3291.012518,NaN,NaN,40,40.0,30
290,NaN,40x40/30x90,3000.0,8.3,1749.76,9.1,TOTAL,3271.537623,3291.012518,32,0,40,40.0,30
307,13096-2,40x40/30x90,3000.0,8.3,1749.76,9.1,32,3271.537623,3291.012518,167,NaN,40,40.0,30
308,NaN,40x40/30x90,3000.0,8.3,1880.992,9.1,0,3271.537623,3291.012518,135,NaN,40,40.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,NaN,40x40/30x90,3000.0,8.3,1880.992,9.1,1661,3271.537623,3291.012518,1661,NaN,40,40.0,30
342,NaN,40x40/30x90,3000.0,8.3,3630.752,9.1,TOTAL,3271.537623,3291.012518,3170,4,40,40.0,30
346,13096-2,40x40/30x90,3000.0,8.3,1749.76,9.1,1509,3271.537623,3291.012518,1509,NaN,40,40.0,30
347,NaN,40x40/30x90,3000.0,8.3,1880.992,9.1,1661,3271.537623,3291.012518,1661,NaN,40,40.0,30


In [212]:
'40+40/2/40/110x80'.split('/')[-1].split('x')[1]

'80'

In [206]:
data['Construction']

1      40+40/2/40/110x80
2      40+40/2/40/110x80
3      40+40/2/40/110x80
4      40+40/2/40/110x80
5      40+40/2/40/110x80
             ...        
663         40x40/100x90
664         40x40/100x90
665         40x40/100x90
666         40x40/100x90
667         40x40/100x90
Name: Construction, Length: 121148, dtype: object

In [87]:
if '+' in '40+40/2/40/110x80'.split('/')[0]:
    warp_count = 'double'
elif 'x' in x.split('/')[0]

'40+40'

In [108]:
'40+40/2/40/110x80'.split('/')

['40+40', '2', '40', '110x80']

In [110]:
'40+40/2/40/110x80'.split('/')[2]

'40'

In [169]:
data.to_csv('all_months_data_2022.csv', index=False)

In [178]:
data['warp_count'] = data['warp_count'].replace(40100, 100)

In [183]:
data['warp_count'] = data['warp_count'].replace(40120, 120)

In [197]:
def warp_count_clean(x):
    if x == 501:
        return 50
    elif x== 40100:
        return 100
    elif x == 40140:
        return 140
    elif x == 40120:
        return 120
    elif x== 501:
        return 50
    else:
        return x

In [198]:
data['weft_count'].apply(lambda x: warp_count_clean(x)).value_counts()

40.0     101589
50.0      10730
30.0       4125
60.0       1789
20.0       1497
100.0       259
80.0        242
120.0        60
140.0        30
70.0         21
Name: weft_count, dtype: int64

In [200]:
data['weft_count'] = data['weft_count'].apply(lambda x: warp_count_clean(x))

In [203]:
data.to_csv('Mega_data.csv', index=False)

In [202]:
# data['weft_count'].value_counts()

In [151]:
# def construction_weft_count(x):
#     try: 
#         if '+' in x.split('/')[0]:
#             return x.split('/')[2]
#         elif 'x' in x.split('/')[0]:
#             return int(x.split('/')[0].split('x')[1])
        
#     except :
#         if 'z' in x.split('/')[0]:
#             return x.split('/')[0].split('x')[-1][-2:]
#         elif 'x' in x.split('/')
#         else:
#             return x
        

In [129]:
'40xz40/120x80'.split('/')[0]

'40xz40'

In [145]:
'40xz40/120x80'.split('/')[0].split('x')[-1][-2:]

'40'

In [137]:
if 'z' in '40xz40/120x80'.split('/')[0]:
    print('40xz40/120x80'.split('/').split('x')   )

AttributeError: 'list' object has no attribute 'split'

In [155]:
data['Construction'].apply(lambda x: construction_weft_count(x)).value_counts()

40                  100505
50                   10637
30                    4023
60                    1747
20                    1371
40                     873
110x80                 514
100x60                 102
501                     75
40100                   63
40120                   60
100x70                  48
60+40x60/130x100        42
110x84                  36
70                      21
156x84                  18
Name: Construction, dtype: int64

In [159]:
data['Construction'].apply(lambda x: construction_weft(x)).value_counts()

40.0       101589
50.0        10655
30.0         4125
60.0         1789
20.0         1497
80.0          242
100.0         196
501.0          75
40100.0        63
40120.0        60
40140.0        30
70.0           21
Name: Construction, dtype: int64

In [95]:
'40+40/2/40/110x80'.split('/')

['40+40', '2', '40', '110x80']

In [83]:
def construction_weft_count(x):
        if '+' in x.split('/')[0]:
            return x.split('/')[-1]
        else:
            return x.split('/')[0][-2:]

In [84]:
data['Construction'].apply(lambda x: construction_weft_count(x))

1      110x80
2      110x80
3      110x80
4      110x80
5      110x80
        ...  
663        40
664        40
665        40
666        40
667        40
Name: Construction, Length: 121148, dtype: object

In [71]:
def construction_break(x):
    x1 = x.split('/')[0]
    ppi= x.split('/')[-1].split('x')[-1]
    epi= x.split('/')[-1].split('x')[0]
    try:
        if '+' in x1:
            warp_count = 'double'
    except:
        warp_count = int(x1[:2])
        weft_count = int(x1[3:])
    return x1, epi, ppi, warp_count, weft_count

In [77]:
def construction_warp_count(x):
    x1 = x.split('/')[0]
    try:
        if '+' in x1:
            warp_count = 'double'
    except:
        try:
            warp_count = int(x1[:2])
        except:
            return x1
    return warp_count

In [78]:
data['Construction'].apply(lambda x: construction_warp_count(x))

UnboundLocalError: local variable 'warp_count' referenced before assignment

In [45]:
data['Construction'].value_counts()

40x40/110x90       40791
40x40/120x80       15589
40x40/120x90       11768
40x40/130x80        9893
40x40/110x70        6975
                   ...  
40x30/2x/100x50        9
60x50/150x96           6
60x60/156x/100         6
470x40/120x80          3
40xz40/100x70          3
Name: Construction, Length: 128, dtype: int64

In [41]:
data.shape

(121148, 11)

In [27]:
# april = pd.read_excel('april-13.xlsx', header=[0,1])